In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LiDARSCRNet(nn.Module):
    def __init__(self, input_channels=2, num_classes=2, dropout_rate=0.2):
        super(LiDARSCRNet, self).__init__()

        # Stack 1
        self.conv1 = nn.Conv2d(input_channels, 4, kernel_size=(1, 5), padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 2))

        # Stack 2
        self.conv2 = nn.Conv2d(4, 4, kernel_size=(1, 5), padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=(1, 2))

        # Stack 3
        self.conv3 = nn.Conv2d(4, 8, kernel_size=(1, 5), padding=0)
        self.pool3 = nn.MaxPool2d(kernel_size=(1, 2))

        # Fusion Stack
        self.fusion_conv1 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.fusion_pool1 = nn.MaxPool2d(kernel_size=(2, 3))

        self.fusion_conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.fusion_pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        # Final classifier
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(576, num_classes)

    def forward(self, x):
        # x: (B, 2, 460, 16) or (B, 2, 216, 16) after permuting dims from (B, 16, 460, 2)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = F.relu(self.fusion_conv1(x))
        x = self.fusion_pool1(x)

        x = F.relu(self.fusion_conv2(x))
        x = self.fusion_pool2(x)

        x = torch.flatten(x, 1)  # flatten except batch
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [2]:
class mmWaveSCRNet(nn.Module):
    def __init__(self, input_channels=1, num_classes=2, dropout_rate=0.2):
        super(mmWaveSCRNet, self).__init__()

        # Stack 1
        self.conv1 = nn.Conv2d(input_channels, 4, kernel_size=(1, 3), padding=(0, 1))
        
        # Stack 2
        self.conv2 = nn.Conv2d(4, 8, kernel_size=(1, 3), padding=(0, 1))

        # Fusion Stack
        self.fusion_conv1 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.fusion_pool1 = nn.MaxPool2d(kernel_size=(2, 3))

        self.fusion_conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.fusion_pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        # Fully connected output layer
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(576, num_classes)

    def forward(self, x):
        # x: (B, 1, 1, 54)
        x = F.relu(self.conv1(x))  # -> (B, 4, 1, 54)
        x = F.relu(self.conv2(x))  # -> (B, 8, 1, 54)

        x = F.relu(self.fusion_conv1(x))  # -> (B, 16, 1, 54)
        x = self.fusion_pool1(x)          # -> (B, 16, ?, ?)

        x = F.relu(self.fusion_conv2(x))  # -> (B, 16, ?, ?)
        x = self.fusion_pool2(x)          # -> (B, 16, ?, ?)

        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [3]:
import pandas as pd


dataset_csv = pd.read_csv("../Carla/datasets/scenario25_dev/scenario25.csv")

In [4]:
dataset_csv.head()

,index,unit1_rgb,unit1_pwr_60ghz,unit1_lidar,unit1_lidar_SCR,unit1_blockage,unit1_loc,unit2_loc,time_stamp[UTC],seq_index
0,1,./unit1/camera_data/image1_08_18_30.jpg,./unit1/mmWave_data/mmWave_power_1.txt,./unit1/LiDAR_data/lidar_data_1.mat,./unit1/LiDAR_SCR_data/lidar_SCR_data_1.mat,./unit1/label_data/label_1.txt,./unit1/GPS_data/gps_location.txt,./unit2/GPS_data/gps_location.txt,['08-18-30-0'],1
1,2,./unit1/camera_data/image2_08_18_31.jpg,./unit1/mmWave_data/mmWave_power_2.txt,./unit1/LiDAR_data/lidar_data_2.mat,./unit1/LiDAR_SCR_data/lidar_SCR_data_2.mat,./unit1/label_data/label_2.txt,./unit1/GPS_data/gps_location.txt,./unit2/GPS_data/gps_location.txt,['08-18-31-0'],1
2,3,./unit1/camera_data/image3_08_18_31.jpg,./unit1/mmWave_data/mmWave_power_3.txt,./unit1/LiDAR_data/lidar_data_3.mat,./unit1/LiDAR_SCR_data/lidar_SCR_data_3.mat,./unit1/label_data/label_3.txt,./unit1/GPS_data/gps_location.txt,./unit2/GPS_data/gps_location.txt,['08-18-31-142'],1
3,4,./unit1/camera_data/image4_08_18_31.jpg,./unit1/mmWave_data/mmWave_power_4.txt,./unit1/LiDAR_data/lidar_data_4.mat,./unit1/LiDAR_SCR_data/lidar_SCR_data_4.mat,./unit1/label_data/label_4.txt,./unit1/GPS_data/gps_location.txt,./unit2/GPS_data/gps_location.txt,['08-18-31-284'],1
4,5,./unit1/camera_data/image5_08_18_31.jpg,./unit1/mmWave_data/mmWave_power_5.txt,./unit1/LiDAR_data/lidar_data_5.mat,./unit1/LiDAR_SCR_data/lidar_SCR_data_5.mat,./unit1/label_data/label_5.txt,./unit1/GPS_data/gps_location.txt,./unit2/GPS_data/gps_location.txt,['08-18-31-426'],1


In [5]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Example dataset mock (remove if you're loading from CSV)
# dataset_csv = pd.DataFrame({'time_stamp[UTC]': ["['08-18-30-0']", "['08-18-31-100']", "['08-18-32-200']"]})

# Convert timestamp strings to datetime objects
def parse_timestamp(ts_str):
    # Strip brackets and quotes, then split
    clean_str = ts_str.strip("[]'\" ")
    parts = list(map(int, clean_str.split('-')))
    # Format: hh-mm-ss-ms
    return datetime(2023, 1, 1, parts[0], parts[1], parts[2], parts[3] * 1000)

# Apply conversion
timestamps = dataset_csv['time_stamp[UTC]']
time_objects = [parse_timestamp(ts) for ts in timestamps]

# Compute deltas (in milliseconds)
time_deltas = [
    (time_objects[i+1] - time_objects[i]).total_seconds() * 1000
    for i in range(len(time_objects) - 1)
]

# Plotting
'''
plt.figure(figsize=(12, 4))
plt.plot(time_deltas, marker='o')
plt.title("Time Differences Between Consecutive Samples")
plt.ylabel("Delta Time (ms)")
plt.xlabel("Sample Index")
plt.grid(True)
plt.show()
'''

'\nplt.figure(figsize=(12, 4))\nplt.plot(time_deltas, marker=\'o\')\nplt.title("Time Differences Between Consecutive Samples")\nplt.ylabel("Delta Time (ms)")\nplt.xlabel("Sample Index")\nplt.grid(True)\nplt.show()\n'

In [6]:
print(min(time_deltas), max(time_deltas))

100.0 150000.0


In [7]:
temp = [x for x in zip(time_deltas, range(1, len(time_deltas) + 1)) if x[0] > 5000]

In [8]:
dataset_csv.query("index in (19999, 20000, 20001)")

,index,unit1_rgb,unit1_pwr_60ghz,unit1_lidar,unit1_lidar_SCR,unit1_blockage,unit1_loc,unit2_loc,time_stamp[UTC],seq_index
19998,19999,./unit1/camera_data/image19999_08_55_47.jpg,./unit1/mmWave_data/mmWave_power_19999.txt,./unit1/LiDAR_data/lidar_data_19999.mat,./unit1/LiDAR_SCR_data/lidar_SCR_data_19999.mat,./unit1/label_data/label_19999.txt,./unit1/GPS_data/gps_location.txt,./unit2/GPS_data/gps_location.txt,['08-55-47-664'],1
19999,20000,./unit1/camera_data/image20000_08_55_47.jpg,./unit1/mmWave_data/mmWave_power_20000.txt,./unit1/LiDAR_data/lidar_data_20000.mat,./unit1/LiDAR_SCR_data/lidar_SCR_data_20000.mat,./unit1/label_data/label_20000.txt,./unit1/GPS_data/gps_location.txt,./unit2/GPS_data/gps_location.txt,['08-55-47-830'],1
20000,20001,./unit1/camera_data/image20001_08_57_44.jpg,./unit1/mmWave_data/mmWave_power_20001.txt,./unit1/LiDAR_data/lidar_data_20001.mat,./unit1/LiDAR_SCR_data/lidar_SCR_data_20001.mat,./unit1/label_data/label_20001.txt,./unit1/GPS_data/gps_location.txt,./unit2/GPS_data/gps_location.txt,['08-57-44-0'],1


# Fix the the address issue 

In [9]:
# Base directory where the actual files are located
base_dir = "../Carla/datasets/scenario25_dev/"

# Columns that contain relative paths
path_columns = [
    'unit1_rgb', 'unit1_pwr_60ghz', 'unit1_lidar', 'unit1_lidar_SCR',
    'unit1_blockage', 'unit1_loc', 'unit2_loc'
]

# Update each path column by prepending the base_dir
for col in path_columns:
    dataset_csv[col] = base_dir + dataset_csv[col].astype(str).str.lstrip("./")

# Save to new CSV (optional)
# dataset_csv.to_csv('fixed_dataset.csv', index=False)

# Now dataset_csv has updated relative paths
print(dataset_csv[path_columns].head())

                                           unit1_rgb  \
0  ../Carla/datasets/scenario25_dev/unit1/camera_...   
1  ../Carla/datasets/scenario25_dev/unit1/camera_...   
2  ../Carla/datasets/scenario25_dev/unit1/camera_...   
3  ../Carla/datasets/scenario25_dev/unit1/camera_...   
4  ../Carla/datasets/scenario25_dev/unit1/camera_...   

                                     unit1_pwr_60ghz  \
0  ../Carla/datasets/scenario25_dev/unit1/mmWave_...   
1  ../Carla/datasets/scenario25_dev/unit1/mmWave_...   
2  ../Carla/datasets/scenario25_dev/unit1/mmWave_...   
3  ../Carla/datasets/scenario25_dev/unit1/mmWave_...   
4  ../Carla/datasets/scenario25_dev/unit1/mmWave_...   

                                         unit1_lidar  \
0  ../Carla/datasets/scenario25_dev/unit1/LiDAR_d...   
1  ../Carla/datasets/scenario25_dev/unit1/LiDAR_d...   
2  ../Carla/datasets/scenario25_dev/unit1/LiDAR_d...   
3  ../Carla/datasets/scenario25_dev/unit1/LiDAR_d...   
4  ../Carla/datasets/scenario25_dev/unit1/LiDA

In [10]:
# Load your CSV

# Define a function to read and convert the blockage label
def read_blockage_label(file_path):
    try:
        with open(file_path.strip(), 'r') as f:
            value = f.read().strip()
            return int(float(value) > 0)  # Handles '1', '0', '1.0', '0.0'
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return -1  # or some fallback value

# Apply to each row
dataset_csv['blockage_label'] = dataset_csv['unit1_blockage'].apply(read_blockage_label)

In [11]:
dataset_csv.to_csv("Deepsense_dataset.csv", index=False)